In [ ]:
#! pip install awswrangler

## 1. Upload data to S3

First you need to create a bucket for this experiment. Upload the data from the following public location to your own S3 bucket. To facilitate the work of the crawler use two different prefixs (folders): one for the billing information and one for reseller. 

### Download the data

In [ ]:
# your bucket name
your_bucket = 'XXXXXXXXXXX'

In [ ]:
!wget https://ml-lab-mggaska.s3.amazonaws.com/billing_sm.csv
!wget https://ml-lab-mggaska.s3.amazonaws.com/reseller_sm.csv
!wget https://ml-lab-mggaska.s3.amazonaws.com/awswrangler-0.0b2-py3.6.egg

In [ ]:
import boto3, os
import awswrangler

boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('billing', 'billing_sm.csv')).upload_file('billing_sm.csv')
boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('reseller', 'reseller_sm.csv')).upload_file('reseller_sm.csv')
boto3.Session().resource('s3').Bucket(your_bucket).Object(os.path.join('python', 'awswrangler-0.0b2-py3.6.egg')).upload_file('awswrangler-0.0b2-py3.6.egg')


## 2. Create a Crawler

To use this csv information in the context of a Glue ETL, first we have to create a Glue crawler pointing to the location of each file. The crawler will try to figure out the data types of each column. 

1. Create an IAM role GlueCrawlerRole with the policy AWSGlueServiceRole.
2. Under Services, go to AWS Glue.
3. Under crawlers Add a Crawler : create one pointing to different each S3 locations (one to billing and one to reseller)

    3.1 Fill  a Crawler Name: point a Data Store to specific S3 path, Navigate to your bucket and your folder: /billing, click "Next"
    
    3.2 Specify "Yes" to add a new Data Store and navigate to your bucket and your folder: /reseller, Click "Next" and select "No" when asking for add more Data stores, use an existing IAM role "AWSGlueServiceRole", add database "implementationdb", Click on "Next" and "Finish"
    
    3.3 After the crawler is created select "Run it now".
    

After the crawler run you should see two tables has been added for each folder. You can use Athena to inspect the tables and double check the data has been added properly.

## 3. Execute a query to create a sample View in Athena

In [19]:

session = awswrangler.Session()
query=('CREATE VIEW resellers_sample AS SELECT *'
       'FROM billing where id_reseller '
       'in (select distinct id_reseller from reseller TABLESAMPLE BERNOULLI(10))')

df = session.pandas.read_sql_athena(
    sql=query,
    database="implementationdb",
    max_result_size=1
)